In [ ]:
import os
import numpy as np 
import pandas as pd 
import cv2 as cv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow import keras



test_data = pd.read_csv("../input/petfinder-pawpularity-score/test.csv").to_numpy()
train_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv").to_numpy()

TEST_PATH = '../input/petfinder-pawpularity-score/test'
TRAIN_PATH = '../input/petfinder-pawpularity-score/train'

dimensions = (128, 128, 1)


In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, directory, list_IDs, n_channels=3, batch_size=8, dim=dimensions, shuffle=True):
        'Initialization'
        self.dim = dim
        self.directory = directory
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        
        X = np.empty((self.batch_size, *self.dim))
        y = []
            
        for batch_number, id_temp_entry in enumerate(list_IDs_temp):
            path = self.directory + '/' + id_temp_entry[0] + '.jpg'
                
            image = cv.imread(path, cv.IMREAD_GRAYSCALE)
            image = cv.resize(image, (self.dim[0],self.dim[1]))
            image = image.reshape(dimensions)
            
            X[batch_number] = image
            
            if len(id_temp_entry) >= 14:
                y.append(id_temp_entry[13])
            
        y = np.array(y)
        if len(y) == 0:
            return X, []
        
        
        return X,  y
                     
    def __getitem__(self, index):
            
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

            
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

            
        X, y = self.__data_generation(list_IDs_temp)
        
        if len(y) == 0:
            return X

        return X, y



In [ ]:
model = Sequential()


model.add(keras.Input(shape=dimensions))
# model.add(keras.layers.RandomRotation(
#     0.15, fill_mode='reflect', interpolation='bilinear',
#     seed=None, fill_value=0.0
# )
# )


model.add(Conv2D(filters=128,padding="same",kernel_size=(15,15)))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))


model.add(Conv2D(filters=128,padding="same",kernel_size=(11,11)))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))



model.add(Flatten())
model.add(Dense(16))
model.add(Activation("relu"))

model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(16))
model.add(Activation("relu"))


# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation("relu"))

# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation("relu"))


# model.add(Flatten())
# model.add(Dense(128))
# model.add(Activation("relu"))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation("relu"))


# model.add(Flatten())
# model.add(Dense(32))
# model.add(Activation("relu"))

# model.add(Flatten())
# model.add(Dense(16))
# model.add(Activation("relu"))

# model.add(Flatten())
# model.add(Dense(8))
# model.add(Activation("relu"))


model.add(Flatten())
model.add(Dense(1))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=100000,
    decay_rate=1.2,
    staircase=True)

opt = SGD(learning_rate=lr_schedule)
loss_fn = keras.losses.MeanAbsoluteError()
model.compile(loss=loss_fn,optimizer=opt,metrics=['mae', 'mse'])

In [ ]:


training_generator = DataGenerator(TRAIN_PATH, train_data, batch_size=64)
validation_generator = DataGenerator(TEST_PATH, test_data, batch_size=1)

model.fit(training_generator,
                    validation_data=validation_generator, epochs = 10)
model.summary()
results = model.predict(validation_generator)



In [ ]:
submission = []

for index, result in enumerate(results):
    submission.append([test_data[index][0], float(result[0])])
    
    
df = pd.DataFrame(submission, columns=['Id', 'Pawpularity'])
df = df.sort_values('Id')
df.to_csv('submission.csv', index=False)
df